In [123]:


import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.math as afwMath
import lsst.afw.detection as afwDetection
import lsst.afw.display.utils as displayUtils
import lsst.afw.table as afwTable
import lsst.meas.algorithms as measAlg
import lsst.afw.display.ds9 as ds9
import numpy as np
import os

Working directory plus transparence setting for ds9

In [124]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/data/CFHTLS"

calexp_1 = DATADIR+'/output/subtracted+1/33,15.fits'
ds9.setMaskTransparency(40)

Creation of the new image as an Exposure (image including masks)

In [125]:
exposure = afwImage.ExposureF(calexp_1)
mi = exposure.getMaskedImage()
im = mi.getImage()

MEANCLIP and STDEVCLIP calculations. Mean could mean really "Median". To verify in afwMath.

In [126]:
nsigma = 5
stats = afwMath.makeStatistics(im, afwMath.MEANCLIP | afwMath.STDEVCLIP)
threshold5 = stats.getValue(afwMath.MEANCLIP) + nsigma*stats.getValue(afwMath.STDEVCLIP)
print "Mean of the image is %.3f counts, stdev is %.3f counts" % (stats.getValue(afwMath.MEANCLIP), stats.getValue(afwMath.STDEVCLIP))
print "The %d-sigma detection threshold is at %.2f counts" % (nsigma, threshold5)


Mean of the image is 0.007 counts, stdev is 0.190 counts
The 5-sigma detection threshold is at 0.96 counts


This part starts with the creation of a minimal empty schema to store all the values to be calculated. The second task in order to invoke the tasks is to instanciate the respective config classes for the SourceDetectionTask, SourceMeasuramentTask and the MeasurePsfTask.

This code re-uses the config variable for the three tasks mentioned above. The last one emplois a list of badflags (how are they determinated?) and creates a psfDeterminer. This psfDeterminer seems to calculate/create the kernel associated to the PSF.

In [127]:
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.algorithms.measurement import SourceMeasurementTask
from lsst.pipe.tasks.measurePsf import MeasurePsfTask

schema = afwTable.SourceTable.makeMinimalSchema()

config = SourceDetectionTask.ConfigClass()
config.reEstimateBackground = False
detectionTask = SourceDetectionTask(config=config, schema=schema)
 
config = SourceMeasurementTask.ConfigClass()
config.slots.psfFlux = "flux.sinc" # use of the psf flux is hardcoded in secondMomentStarSelector
measureTask = SourceMeasurementTask(schema, config=config)

config = MeasurePsfTask.ConfigClass()
starSelector = config.starSelector.apply()
starSelector.config.badFlags = ["flags.pixel.edge", "flags.pixel.cr.center", "flags.pixel.interpolated.center", "flags.pixel.saturated.center"]
psfDeterminer = config.psfDeterminer.apply()
psfDeterminer.config.sizeCellX = 64
psfDeterminer.config.sizeCellY = 64
psfDeterminer.config.spatialOrder = 1
psfDeterminer.config.nEigenComponents = 3
measurePsfTask = MeasurePsfTask(config=config, schema=schema)


This is the clearing the detection mask, using and AND operator and a negation of the mask.

In [128]:
# Again clear the detection masks
mask = exposure.getMaskedImage().getMask()
mask &= ~(mask.getPlaneBitMask("DETECTED")) 
frame=1
ds9.mtv(exposure, frame=frame, title="Original Image Off of Disk"); frame += 1

Traceback (most recent call last):
  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/site-packages/IPython/core/ultratb.py", line 759, in structured_traceback
    records = _fixed_getinnerframes(etb, context, tb_offset)
  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/site-packages/IPython/core/ultratb.py", line 242, in _fixed_getinnerframes
    records  = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/inspect.py", line 1043, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/inspect.py", line 1003, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/inspect.py", line 451, in getsourcefile
    if os.path.exists(filename):
  File "/renoir_data_00/jpreyes/stacks/v9_2_copy/Linux64/anaconda/1.8.0/lib/python2.7/genericpath.py", line 18, in exists
    os.stat(path)
KeyboardInterrupt



Unfortunately, your original traceback can not be constructed.



The detectionTask is launched to detect the sources. The measureTask is using the sources and the exposure to -probably- generate all the relevant data.

In [ ]:
# And then re-detect, re-measure, select stars, and build the Psf model
table = afwTable.SourceTable.make(schema)
sources = detectionTask.run(table, exposure, sigma=2).sources
measureTask.measure(exposure, sources)
ds9.mtv(exposure, frame=frame, title="Original Image Off of Disk"); frame += 1

The PSF is measured. An image for the PSF calculated is computed. (What are the parameters? Is it the linear combination of multiple gaussian functions?)

In [ ]:
for source in sources:
    print source.getX(), source.getY()

In [ ]:
ksources = sources[1:]
for source in ksources:
    print source.getRa(), source.getDec()
    
print exposure.getXY0()

wcs= exposure.getWcs()

print wcs.pixelToSky(33593, 15928)

In [ ]:

result = measurePsfTask.run(exposure, ksources)

psf = result.psf
psfIm = psf.computeImage()
ds9.mtv(psfIm, frame=frame, title="Psf Image"); frame += 1

This part creates a mosaic of the psf to be observed via ds9.

In [ ]:
import lsst.meas.algorithms.utils as measUtils
cellSet = result.cellSet
measUtils.showPsfMosaic(exposure, psf=psf, frame=frame); frame += 1

This method paints in ds9 the detected sources. a + is located on the centroid of the psf, an x is set in place if it is a psf candidate and a red circle is drawn if it was used at the end.

In [ ]:
ds9.mtv(exposure, frame=frame, title="Re-Measured Footprints")
with ds9.Buffering():
    for s in sources:
        xy = s.getCentroid()
        ds9.dot('+', *xy, frame=frame)
        if s.get("calib.psf.candidate"):
            ds9.dot('x', *xy, ctype=ds9.YELLOW, frame=frame)
        if s.get("calib.psf.used"):
            ds9.dot('o', *xy, size=30, ctype=ds9.RED, frame=frame)
frame += 1


This part creates the gaussian kernel that models the psf of the whole image. it seems to replicate a 2D-function using two equal 1-Dimensional gaussian functions. 

On the second part, the image is smoothed with this gaussian Kernel (via convolving with the 2D-gaussian).

In [129]:
shape = psf.computeShape()
sigma = shape.getDeterminantRadius()
print "Psf approximate Gaussian sigma = %.3f pixels; FWHM = %.3f pixels" % (sigma, sigma * 2. * np.sqrt(2. * np.log(2.)))
kWidth = (int(sigma * 7 + 0.5) // 2) * 2 + 1 # make sure kernel size is odd
gaussFunc = afwMath.GaussianFunction1D(sigma)
gaussKernel = afwMath.SeparableKernel(kWidth, kWidth, gaussFunc, gaussFunc)

im = exposure.getMaskedImage().getImage()
smoothedIm = im.Factory(im.getDimensions())
afwMath.convolve(smoothedIm, im, gaussKernel)
ds9.mtv(smoothedIm, frame=frame, title="Detection Image (smoothed with appx Psf)"); frame += 1

NameError: name 'psf' is not defined

In [130]:

#
#Smooth image with a
# 1 2 1
# 2 4 2
# 1 2 1
# filter
oneD = afwMath.PolynomialFunction1D([2, 0, -1])
kernel = afwMath.SeparableKernel(3, 3, oneD, oneD)
smoothedIm = im.Factory(im.getDimensions())
afwMath.convolve(smoothedIm, im, kernel)


ds9.mtv(smoothedIm, frame=frame, title="Detection Image (smoothed with a filter)"); frame += 1

Footprints are detected over the Smoothed Image. Then, the footprints are being grown in 1 pixel (which operations was used?). 

In [131]:


npixMin=1
grow=1
isotropic=False
#fs0 = afwDetection.FootprintSet(smoothedIm, afwDetection.Threshold(threshold5), npixMin)
fs0 = afwDetection.FootprintSet(smoothedIm, afwDetection.Threshold(threshold5), npixMin)
fs = afwDetection.FootprintSet(fs0, grow, isotropic)
ds9.mtv(smoothedIm, frame=frame, title="Re-detected and re-grown Footprints")
for footprint in fs.getFootprints():
    
    displayUtils.drawFootprint(footprint, frame=frame)
#frame += 1


In [132]:
for footprint in fs.getFootprints():
        xs = footprint.getCentroid().getX()-exposure.getX0()
        ys = footprint.getCentroid().getY()-exposure.getY0()
        #print source.getCentroid().getX(),exposure.getX0()
        ds9.dot("+", xs, ys, frame=frame, size=5, ctype = ds9.RED)

This is the part that measures the centroid and the ApFlux of the detected sources and then stores them on the schema created at the beginning. It seems to emploi a 3 pixel aperture radius by default (why?)

In [133]:
apRad=3
# Define the measurements we want to make
ctrlCentroid = measAlg.SdssCentroidControl()
ctrlAperture = measAlg.SincFluxControl()
ctrlAperture.radius2 = apRad

schema = afwTable.SourceTable.makeMinimalSchema()
algorithms = [
    measAlg.MeasureSourcesBuilder().addAlgorithm(ctrlCentroid).build(schema),
    measAlg.MeasureSourcesBuilder().addAlgorithm(ctrlAperture).build(schema)]

This part creates the columns for the propoerties centroid and flux (sdss seems to be the algorithm implementation for sdsss images).  After using the footprints generated above, the algorithms are applied to each source.

In [134]:
# Build a new, empty source catalog from this schema
cat = afwTable.SourceCatalog(schema)
table = cat.table
table.defineCentroid("centroid.sdss")
table.defineApFlux("flux.sinc")

# Turn the Footprints into Sources 
fs.makeSources(cat)
print "Measuring %d objects" % (len(cat))

# And apply each algorithm to each Source, measuring on the orignal image
for source in cat:
    for alg in algorithms:
        alg.apply(source, exposure)

Measuring 10 objects


In [135]:
ds9.mtv(mi, frame=frame, title="Measured sources",)
ds9.setMaskTransparency(80)
with ds9.Buffering():
    for source in cat:
        xs = source.getCentroid().getX()-exposure.getX0()
        ys = source.getCentroid().getY()-exposure.getY0()
        #print source.getCentroid().getX(),exposure.getX0()
        ds9.dot("+", xs, ys, frame=frame, ctype = ds9.RED)
frame+=1
                
                

In [136]:
for source in cat:
    print source.getApFlux()

3147.52809152
2685.65130901
186.658658656
10.3170599769
191.005948894
10.8447671363
46.072414535
10.0073937245
-5.25992043901
139.226507024
